In [ ]:
from scipy.spatial.distance import cdist
from scipy.spatial import KDTree
from metavision_core.event_io import EventsIterator
from metavision_sdk_cv import TimeGradientFlowAlgorithm, ActivityNoiseFilterAlgorithm
import numpy as np

def group_events(events, r):
    '''
    Group events into clusters based on their spatial proximity.
    '''
    centers = np.array(events[['center_x', 'center_y']].tolist(), dtype=np.float32)
    if len(centers) == 0:
        return np.array([])
    elif len(centers) == 1:
        return np.array([events])
    distance_matrix = cdist(centers, centers, metric='euclidean') # Compute pairwise distances for all events
    groups = []
    grouped = np.zeros(len(events), dtype=bool)
    for i in range(len(events)):
        if not grouped[i]:
            group = np.where(distance_matrix[i] < r) # Events within r pixels are considered to be in the same group
            avg_vx = np.mean(events['vx'][group])
            avg_vy = np.mean(events['vy'][group])
            events[group]['vx'] = avg_vx
            events[group]['vy'] = avg_vy
            groups.append(events[group][0])
            grouped[group] = True
    
    # anything not in a group should be considered as a group of its own
    for i in range(len(events)):
        if not grouped[i]:
            groups.append(events[[i]])
    
    return np.array(groups)

In [ ]:
%matplotlib ipympl

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


# Constants
dt = 200
start_ts = 1
tolerance = 5 # Tolerance for position matching in pixels

# Input for the file path
filename = input('Path to the file: ')
evts_iterator = EventsIterator(filename)
reader = evts_iterator.reader
# Get start timestamp from external trigger
for evts in evts_iterator:
    external_events = evts_iterator.get_ext_trigger_events()
    if len(external_events) > 0:
        start_ts = external_events[0]['t']
        break



# Initialize iterators and algorithms
start_ts = int(start_ts / dt) * dt
evts_iterator = EventsIterator(filename, start_ts=start_ts, relative_timestamps=False, delta_t=dt)
height, width = evts_iterator.get_size()
activity_filter = ActivityNoiseFilterAlgorithm(width, height, 1000)  # 1ms threshold
flow = TimeGradientFlowAlgorithm(width, height, radius=15, bit_cut=5, min_flow_mag=10)
activity_buffer = activity_filter.get_empty_output_buffer()
flow_buffer = flow.get_empty_output_buffer()


# Prepare to store event data
event_data_list = []
query_points_list = []

# Process events
for evts in evts_iterator:
    activity_filter.process_events(evts, activity_buffer)
    flow.process_events(activity_buffer, flow_buffer)
    processed_events = flow_buffer.numpy()
    grouped_events = group_events(processed_events, tolerance)
    current_ts = evts_iterator.get_current_time()
    if len(grouped_events) > 0:
        # Convert structured array to regular NumPy array
        event_data = np.array(processed_events[['center_x', 'center_y', 't']].tolist(), dtype=np.float64)
        event_data_list.append(event_data)


# Concatenate all collected data into single arrays
if event_data_list:
    X = np.concatenate(event_data_list, axis=0)
else:
    X = np.empty((0, 3), dtype=np.float64)

if query_points_list:
    query_points = np.concatenate(query_points_list, axis=0)
else:
    query_points = np.empty((0, 3), dtype=np.float64)

# Create a KDTree for the events
tree = KDTree(X)


